In [6]:
import pandas as pd
import numpy as np
import requests
import time
from googleapiclient.discovery import build
from IPython.display import JSON

In [13]:
api_key = ""

# get credentials and create an API client
youtube = build("youtube", "v3", developerKey=api_key)

In [14]:
channel_ids = ["UCG8rbF3g2AMX70yOd8vqIZg", # Logan Paul
               "UCLhvu_-py2eiC_yD17WLLNg", # Learn & Play With Zack
               "UCAgx4HcQIYn9lM0rhtIuH9w", # HZHtube Fun Kids
              ]

In [15]:
request = youtube.channels().list(
        part="snippet,contentDetails,statistics,topicDetails,status",
        id=','.join(channel_ids)
)
response = request.execute()

In [16]:
JSON(response)

<IPython.core.display.JSON object>

In [17]:
def get_channel_data(youtube, channel_ids):
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics,topicDetails,status",
        id=','.join(channel_ids)
    )
    response = request.execute()
    
    all_data = []
    
    for i in range(len(channel_ids)):
    
        channel = response['items'][i]

        channel_data = {
            'channel_id': channel['id'],
            'channel_name': channel['snippet']['title'],
            'channel_desc': channel['snippet']['description'],
            'channel_start_date': channel['snippet']['publishedAt'],
            'total_views': channel['statistics']['viewCount'],
            'total_subscribers': channel['statistics']['subscriberCount'],
            'total_videos_posted': channel['statistics']['videoCount']
        }
        
        try:
            channel_data['channel_topic'] = channel['topicDetails']['topicCategories']
        except:
            channel_data['channel_topic'] = None
        
        all_data.append(channel_data)
        
    return pd.DataFrame(all_data)
        

In [18]:
get_channel_data(youtube, channel_ids)

,channel_id,channel_name,channel_desc,channel_start_date,total_views,total_subscribers,total_videos_posted,channel_topic
0,UCLhvu_-py2eiC_yD17WLLNg,Heidi Zidane and Zack,Welcome to Heidi Zidane and Zack Youtube Chann...,2018-03-23T14:32:26Z,203726997,1360000,373,[https://en.wikipedia.org/wiki/Lifestyle_(soci...
1,UCAgx4HcQIYn9lM0rhtIuH9w,Heidi and Zidane HZHtube,,2015-11-24T19:25:04Z,3998110674,12400000,750,[https://en.wikipedia.org/wiki/Lifestyle_(soci...
2,UCG8rbF3g2AMX70yOd8vqIZg,Logan Paul,27 year old in Puerto Rico living life to the ...,2015-08-29T03:28:33Z,5972451029,23600000,718,[https://en.wikipedia.org/wiki/Lifestyle_(soci...


### Most Popular Videos

In [24]:
def get_most_popular_videos(youtube):
    
    request = youtube.videos().list(
        part="snippet,contentDetails,status,statistics,topicDetails",
        chart="mostPopular",
        regionCode="US",
        maxResults = 50
    )
    response = request.execute()

    all_data = []

    for i in range(50):
        video = response['items'][i]

        video_data = {
            'video_id': video['id'],
            'channel_id': video['snippet']['channelId'],
            'video_posting_date': video['snippet']['publishedAt'],
            'video_title': video['snippet']['title'],
            'video_desc': video['snippet']['description'],
            'video_length': video['contentDetails']['duration'],
            'made_for_kids': video['status']['madeForKids'],
            'video_views': video['statistics']['viewCount'],
            'video_comments': video['statistics']['commentCount']
        }

        try:
            video_data['video_topic'] = video['topicDetails']['topicCategories']
            video_data['video_likes'] = video['statistics']['likeCount']
        except:
            video_data['video_topic'] = None
            video_data['video_likes'] = None
        all_data.append(video_data)
    
    return pd.DataFrame(all_data).sort_values(by='video_posting_date', ascending=False)

In [25]:
popular_videos = get_most_popular_videos(youtube)
popular_videos.head()

,video_id,channel_id,video_posting_date,video_title,video_desc,video_length,made_for_kids,video_views,video_comments,video_topic,video_likes
15,beSIRUJPEjo,UCv3Kk__aFcKKx_o2686uQcw,2023-04-27T16:00:07Z,"""Blue Lies"" | Clock 0ut Animation",Hi! This one's a bit shorter than the other tw...,PT1M17S,False,162995,1842,[https://en.wikipedia.org/wiki/Music],49593
8,gSDtNkKPiDg,UCp68_FLety0O-n9QU6phsgw,2023-04-27T15:00:34Z,The WEIGHTLESS Machine,New to Warframe? You can play The Duviri Parad...,PT10M13S,False,779550,4256,None,None
27,aOTz0Ol8fLA,UCNSMdQtn1SuFzCZjfK2C7dQ,2023-04-27T15:00:01Z,The Pros and Cons of Every Motorcycle Engine Type,Get a quote on pay-per-mile motorcycle insuran...,PT15M23S,False,181254,2252,[https://en.wikipedia.org/wiki/Lifestyle_(soci...,26961
3,6emgiatRGu4,UCaO6TYtlC8U5ttz62hTrZgg,2023-04-27T14:59:59Z,"Stray Kids ""★★★★★ (5-STAR)"" Trailer","Stray Kids(스트레이 키즈) ""★★★★★ (5-STAR)"" Trailer\n...",PT2M51S,False,1707947,34463,"[https://en.wikipedia.org/wiki/Entertainment, ...",488706
4,SlSfr6Wa5sc,UC_ntXHv-XdKCD7CPynVvnQw,2023-04-27T14:00:14Z,ARMORED CORE VI FIRES OF RUBICON — Gameplay Tr...,"""Augmented human C4-621 has awakened.""\n\nARMO...",PT3M7S,False,461503,2972,[https://en.wikipedia.org/wiki/Action-adventur...,31380
